<a href="https://colab.research.google.com/github/ISHA252001/learningPython/blob/main/Charge_prediction_using_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install rdkit-pypi

     |████████████████████████████████| 20.6 MB 1.3 MB/s 


In [ ]:
import keras as k
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization
from keras.layers.core import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from rdkit import Chem, DataStructs
from rdkit.Chem import rdMolDescriptors, Draw
from rdkit.Chem.Draw import IPythonConsole
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
import math

""" TRAINING DATA PREPROCESSING """
#Convert training data, in the form of txt files of line-by-line SMILES strings and charges into arrays
with open('1K_SMILES_strings.txt') as my_file:
    SMILES_array = my_file.readlines()

with open('1K_SMILES_strings_charges.txt') as my_file:
    charges_array = my_file.readlines()

#Convert testing data, in the form of txt file of line-by-line SMILES strings into arrays
with open('10_SMILES_strings_test.txt') as my_file:
    test_SMILES_array = my_file.readlines()

#Convert each item of the training array of SMILES strings into molecules
mols = [Chem.rdmolfiles.MolFromSmiles(SMILES_string) for SMILES_string in SMILES_array]

#Convert training molecules into training fingerprints
bi = {}
fps = [rdMolDescriptors.GetMorganFingerprintAsBitVect(m, radius=2, bitInfo= bi, nBits=256) for m in mols]

#Convert training fingerprints into binary, and put all training binaries into arrays
np_fps_array = []
for fp in fps:
  arr = np.zeros((1,), dtype= int)
  DataStructs.ConvertToNumpyArray(fp, arr)
  np_fps_array.append(arr)

""" TESTING DATA PREPROCESSING """
#Convert each item of the testing array of SMILES strings into molecules
test_mols = [Chem.rdmolfiles.MolFromSmiles(test_SMILES_string) for test_SMILES_string in test_SMILES_array]

#Convert testing molecules into testing fingerprints
test_fps = [rdMolDescriptors.GetMorganFingerprintAsBitVect(test_m, radius=2, nBits=256) for test_m in test_mols]

#Convert testing fingerprints into binary, and put all testing binaries into arrays
test_np_fps_array = []
for test_fp in test_fps:
  test_arr = np.zeros((1,), dtype= int)
  DataStructs.ConvertToNumpyArray(test_fp, test_arr)
  test_np_fps_array.append(test_arr)
  

"""NEURAL NETWORK"""
#The neural network model
model = Sequential([
    Dense(256, input_shape=(256,), activation= "relu"),
    Dense(128, activation= "sigmoid"),
    Dense(64, activation= "sigmoid"),
    Dense(34, activation= "sigmoid"),
    Dense(16, activation= "sigmoid"),
    BatchNormalization(axis=1),
    Dense(4, activation= "softmax")
])
model.summary()

#Compiling the model
model.compile(optimizer=Adam(lr=0.00001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

print(np.array(np_fps_array))
print(np.array(charges_array))

charges_array_int=[]
for i in range(len(charges_array)):
  charges_array_int.append(int(charges_array[i][0]))

#Training the model
model.fit(np.array(np_fps_array), np.array(charges_array_int), validation_split=0.1, batch_size=10, epochs= 100, shuffle=True, verbose=1)

#Predictions with test dataset
predictions = model.predict(np.array(test_np_fps_array), batch_size=1, verbose=1)

print('class\t0\t1\t\t2\t\t3')
print('charge\t0\t+1\t\t-1\t\t-2')
charges = [0,1,-1,-2]
for prediction in predictions:
    print (prediction, "      Charge: ", charges[np.argmax(prediction)])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 256)               65792     
                                                                 
 dense_13 (Dense)            (None, 128)               32896     
                                                                 
 dense_14 (Dense)            (None, 64)                8256      
                                                                 
 dense_15 (Dense)            (None, 34)                2210      
                                                                 
 dense_16 (Dense)            (None, 16)                560       
                                                                 
 batch_normalization_2 (Batc  (None, 16)               64        
 hNormalization)                                                 
                                                      

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


81/81 [==============================] - 1s 5ms/step - loss: 1.3848 - accuracy: 0.2593 - val_loss: 1.1979 - val_accuracy: 0.5889
Epoch 2/100
81/81 [==============================] - 0s 3ms/step - loss: 1.3801 - accuracy: 0.3247 - val_loss: 1.2255 - val_accuracy: 0.5889
Epoch 3/100
81/81 [==============================] - 0s 3ms/step - loss: 1.3760 - accuracy: 0.3494 - val_loss: 1.2601 - val_accuracy: 0.5889
Epoch 4/100
81/81 [==============================] - 0s 3ms/step - loss: 1.3715 - accuracy: 0.4407 - val_loss: 1.2912 - val_accuracy: 0.5889
Epoch 5/100
81/81 [==============================] - 0s 4ms/step - loss: 1.3672 - accuracy: 0.4864 - val_loss: 1.3153 - val_accuracy: 0.5889
Epoch 6/100
81/81 [==============================] - 0s 3ms/step - loss: 1.3609 - accuracy: 0.5321 - val_loss: 1.3321 - val_accuracy: 0.5889
Epoch 7/100
81/81 [==============================] - 0s 3ms/step - loss: 1.3564 - accuracy: 0.5778 - val_loss: 1.3421 - val_accuracy: 0.6111
Epoch 8/100
81/81 [======